In [1]:
import os
import pickle

import torch
from torch.utils.data import DataLoader

import vae_model.build

In [2]:
config_dir = './vae_model'

# Create model
with open(os.path.join(config_dir, 'subset_samplers.pickle'), 'rb') as f:
    idx_helper = pickle.load(f)
with open(os.path.join(config_dir, 'config_train.pickle'), 'rb') as f:
    conf_train = pickle.load(f)
with open(os.path.join(config_dir, 'config_model.pickle'), 'rb') as f:
    conf_model = pickle.load(f)

In [3]:
_, _, _, model = vae_model.build.build_extended_ae_model(conf_model, conf_train,
                                                                 idx_helper)
model = model.eval()

FORCE BIGGER ENC/DEC NETWORKS = False
MIDI NOTES = ((60, 100),)
self.spectrogram_input_size (257, 347)


In [4]:
# Load from checkpoint
checkpoint_state_dict = torch.load(
    os.path.join(config_dir, '00133.tar'), 
    map_location=torch.device('cpu')
)

In [5]:
model.load_state_dict(checkpoint_state_dict['ae_model_state_dict'])

<All keys matched successfully>

In [6]:
ae_model, reg_model = model.ae_model, model.reg_model

In [7]:
torch.set_grad_enabled(False)

In [8]:
# This would come from the Dataset
sample_spectrogram_input = torch.randn(1, 1, 257, 347)

preset_UID = 0
ref_midi_pitch = 60
ref_midi_velocity = 100

sample_info = torch.tensor([preset_UID, ref_midi_pitch, ref_midi_velocity], dtype=torch.int32).unsqueeze(0)

In [9]:
ae_out = ae_model(sample_spectrogram_input, sample_info)  # Spectral VAE - tuple output

In [10]:
len(ae_out)

5

In [11]:
_, _, z_K_sampled, _, _ = ae_out
v_out = reg_model(z_K_sampled)

In [12]:
v_out.shape

torch.Size([1, 610])

### This `v_out` tensor is what's being used to get the full dexed synth params

In [13]:
from data.preset import DexedPresetsParams
import vae_config
import data.build

Can't import sounddevice!!!!


In [14]:
model_config, train_config = vae_config.model, vae_config.train

In [15]:
dataset = data.build.get_dataset(model_config, train_config)

[PresetDataset] Cannot open '/home/ubuntu/environment/project/audio-synth-rl/data/stats/DexedDataset_spectrogram_nfft1024hop0256mels0257.json' stats file.
[PresetDataset] No pre-computed spectrogram stats can be found. No normalization will be performed
[PresetIndexesHelper] 144 learnable VSTi parameters, learnable tensor representation size: 610


In [16]:
out_presets_instance = DexedPresetsParams(learnable_presets=v_out, dataset=dataset)

In [18]:
out_presets_instance.get_full().shape

torch.Size([1, 155])